In [2]:
import numpy as np
from scipy.stats import norm
rng = np.random.default_rng(7)

# -------------------------------------------------
# Inputs
# -------------------------------------------------
S0, K = 100, 100              # spot, strike
r, σ  = 0.02, 0.20            # risk-free, volatility
T, steps = 1.0, 252           # years, hedge dates
N = 20_000                    # Monte-Carlo paths
dt = T / steps

# -------------------------------------------------
# Helper: Black–Scholes Δ (vectorised)
# -------------------------------------------------
def delta_bs(S, τ):
    """Δ of a call; S and τ can be NumPy arrays (broadcasts)."""
    sqrtτ = np.sqrt(τ)
    d1 = (np.log(S / K) + (r + 0.5*σ**2)*τ) / (σ*sqrtτ)
    return norm.cdf(d1)

# Closed-form option price at t=0
d1_0 = (np.log(S0/K)+(r+0.5*σ**2)*T)/(σ*np.sqrt(T))
d2_0 = d1_0 - σ*np.sqrt(T)
C0   = S0*norm.cdf(d1_0) - K*np.exp(-r*T)*norm.cdf(d2_0)

# -------------------------------------------------
# 1. Simulate all price paths in one matrix  (N × steps)
# -------------------------------------------------
Z   = rng.standard_normal((N, steps))
logS = (np.log(S0) +
        np.cumsum((r-0.5*σ**2)*dt + σ*np.sqrt(dt)*Z, axis=1))
S    = np.hstack((np.full((N,1), S0), np.exp(logS)))   # N × (steps+1)

# -------------------------------------------------
# 2. Pre-compute all deltas (same shape)
# -------------------------------------------------
τ    = T - np.arange(steps+1)*dt            # time-to-mat vector, length steps+1
Δ    = delta_bs(S, τ)                       # broadcasts to N × (steps+1)

# -------------------------------------------------
# 3. Vectorised hedging cash-account
# -------------------------------------------------
cash      = C0 - Δ[:,0]*S[:,0]              # N-vector
Δ_prev    = Δ[:,0]

for k in range(1, steps+1):
    cash *= np.exp(r*dt)                    # grow at r
    dΔ     = Δ[:,k] - Δ_prev
    cash  -= dΔ * S[:,k]                    # trade stock
    Δ_prev = Δ[:,k]

# -------------------------------------------------
# 4. Hedge error statistics
# -------------------------------------------------
portfolio = cash + Δ_prev * S[:,-1]
payoff    = np.maximum(S[:,-1]-K, 0.0)
errors    = portfolio - payoff

print(f"mean error   : {errors.mean(): .4f}")
print(f"st.dev error : {errors.std(ddof=1): .4f}")


/tmp/ipython-input-1132693672.py:20: RuntimeWarning: divide by zero encountered in divide
  d1 = (np.log(S / K) + (r + 0.5*σ**2)*τ) / (σ*sqrtτ)


mean error   : -0.0021
st.dev error :  0.4384


In [ ]:
#Delta hedging: buy or sell a bit of stock so tiny price changes don’t affect your option trade.